<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%A7%D9%84%D9%87%D9%8A%D8%AF%D8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def hedge_position_with_retrace_limit(symbol):
    """تطبيق استراتيجية الهيدج مع قيود على عدد الصفقات المفتوحة وحجم الصفقة الجديدة بنفس الاتجاه."""
    positions = mt5.positions_get(symbol=symbol)
    open_positions_count = len(positions)

    # تحقق من عدد الصفقات المفتوحة
    if open_positions_count >= 7:
        print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7).")
        return

    if positions:
        for position in positions:
            # تحديد سعر الدخول ووقف الخسارة
            entry_price = position.price_open
            stop_loss = position.sl  # سعر وقف الخسارة

            # حساب المسافة بين سعر الدخول ووقف الخسارة
            distance_to_sl = entry_price - stop_loss if position.type == 0 else stop_loss - entry_price  # 0 = BUY, 1 = SELL

            # حساب السعر الذي يمثل 75% من المسافة نحو وقف الخسارة
            retrace_price = entry_price - (0.75 * distance_to_sl) if position.type == 0 else entry_price + (0.75 * distance_to_sl)  # 0 = BUY, 1 = SELL

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask  # 0 = BUY, 1 = SELL

            # إذا كان السعر الحالي قد تراجع بنسبة 75% نحو وقف الخسارة
            if (position.type == 0 and current_price <= retrace_price) or \
               (position.type == 1 and current_price >= retrace_price):  # 0 = BUY, 1 = SELL

                # فتح مركز بنفس الاتجاه
                hedge_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": position.volume,  # نفس حجم المركز الحالي
                    "type": position.type,  # نفس نوع الصفقة (0 = BUY, 1 = SELL)
                    "price": mt5.symbol_info_tick(symbol).ask if position.type == 0 else mt5.symbol_info_tick(symbol).bid,
                    "sl": 0,
                    "tp": 0,
                    "magic": 123456,
                    "comment": "Hedge position in the same direction",
                }

                # إرسال الطلب
                result = mt5.order_send(hedge_request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"فشل في فتح الصفقة: {result.retcode}")


In [ ]:
#هيدج طبيعي
def hedge_position_with_retrace_limit(symbol):
    """تطبيق استراتيجية الهيدج مع قيود على عدد الصفقات المفتوحة وحجم الصفقة الجديدة."""
    positions = mt5.positions_get(symbol=symbol)
    open_positions_count = len(positions)

    # تحقق من عدد الصفقات المفتوحة
    if open_positions_count >= 7:
        print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7).")
        return

    if positions:
        for position in positions:
            # تحديد سعر الدخول ووقف الخسارة
            entry_price = position.price_open
            stop_loss = position.sl  # سعر وقف الخسارة

            # حساب المسافة بين سعر الدخول ووقف الخسارة
            distance_to_sl = entry_price - stop_loss if position.type == 0 else stop_loss - entry_price  # 0 = BUY, 1 = SELL

            # حساب السعر الذي يمثل 75% من المسافة نحو وقف الخسارة
            retrace_price = entry_price - (0.75 * distance_to_sl) if position.type == 0 else entry_price + (0.75 * distance_to_sl)  # 0 = BUY, 1 = SELL

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask  # 0 = BUY, 1 = SELL

            # إذا كان السعر الحالي قد تراجع بنسبة 75% نحو وقف الخسارة
            if (position.type == 0 and current_price <= retrace_price) or \
               (position.type == 1 and current_price >= retrace_price):  # 0 = BUY, 1 = SELL

                # فتح مركز معكوس إذا كان العدد أقل من 7
                hedge_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": position.volume,  # نفس حجم المركز الحالي
                    "type": 1 if position.type == 0 else 0,  # 1 = SELL, 0 = BUY
                    "price": mt5.symbol_info_tick(symbol).ask if position.type == 0 else mt5.symbol_info_tick(symbol).bid,
                    "sl": 0,
                    "tp": 0,
                    "magic": 123456,
                    "comment": "Hedge position",
                }

                # إرسال الطلب
                result = mt5.order_send(hedge_request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"فشل في فتح الصفقة: {result.retcode}")


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def fetch_high_impact_news():
    """جمع الأخبار ذات التأثير العالي على الدولار من المفكرة الاقتصادية لموقع Investing."""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    high_impact_news = []

    # استخراج الأخبار ذات التأثير العالي على الدولار
    for row in soup.find_all('tr'):
        impact = row.find('td', class_='impact')
        if impact and 'high' in impact['class']:
            time = row.find('td', class_='date').text.strip()
            currency = row.find('td', class_='currency').text.strip()
            if currency == 'USD':  # التأكد من أن الخبر متعلق بالدولار
                news_time = datetime.strptime(time, "%d %b %Y %H:%M")
                high_impact_news.append({"time": news_time.strftime("%Y-%m-%d %H:%M:%S"), "impact": "high"})

    return high_impact_news

def is_within_news_timeframe():
    """تحقق مما إذا كان الوقت الحالي ضمن فترة الأخبار."""
    current_time = datetime.now()
    news_events = fetch_high_impact_news()

    for news in news_events:
        news_time = datetime.strptime(news["time"], "%Y-%m-%d %H:%M:%S")
        before_news = news_time - timedelta(minutes=30)
        after_news = news_time + timedelta(minutes=30)

        if before_news <= current_time <= after_news:
            return True  # الوقت الحالي ضمن فترة الأخبار
    return False

def close_profitable_positions_before_after_news(symbol):
    """إغلاق الصفقات الرابحة قبل وبعد صدور الأخبار ذات التأثير العالي."""
    current_time = datetime.now()
    news_events = fetch_high_impact_news()

    for news in news_events:
        news_time = datetime.strptime(news["time"], "%Y-%m-%d %H:%M:%S")
        before_news = news_time - timedelta(minutes=30)
        after_news = news_time + timedelta(minutes=30)

        if before_news <= current_time <= after_news:
            positions = mt5.positions_get(symbol=symbol)
            for position in positions:
                if position.profit > 0:  # إذا كانت الصفقة رابحة
                    request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": position.ticket,
                        "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                        "volume": position.volume,
                        "price": mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask,
                        "deviation": 10,
                        "magic": 234000,
                        "comment": "Closing profitable position before/after high impact news",
                    }
                    result = mt5.order_send(request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"Closed profitable position for {position.ticket}.")

def place_order(symbol, order_type, volume, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size."""
    if is_within_news_timeframe():
        print("لا يمكن فتح صفقات خلال فترة الأخبار ذات التأثير العالي.")
        return  # لا تفتح الصفقة إذا كان الوقت ضمن فترة الأخبار

    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        sl = price - 1.5 * atr_value if order_type == "buy" else price + 1.5 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

# يمكنك إضافة المزيد من الوظائف أو تحسين الاستراتيجية حسب الحاجة


In [ ]:
import MetaTrader5 as mt5

def calculate_lot_size(account_balance, risk_percentage):
    """حساب حجم العقد بناءً على رصيد الحساب ونسبة المخاطرة."""
    risk_amount = account_balance * risk_percentage
    lot_size = risk_amount / 1000  # افتراض أن كل نقطة تمثل 10 وحدات
    return max(0.01, lot_size)  # الحد الأدنى لحجم العقد هو 0.01

def dynamic_hedge(symbol, risk_percentage=0.01):
    """تطبيق استراتيجية الهيدج مع حجم ديناميكي."""
    # محاولة الاتصال بـ MetaTrader 5
    if not mt5.initialize():
        print("فشل في الاتصال بـ MetaTrader 5:", mt5.last_error())
        return

    try:
        # الحصول على معلومات الحساب
        account_info = mt5.account_info()
        if account_info is None:
            print("فشل في استرداد معلومات الحساب.")
            return

        account_balance = account_info.balance  # الحصول على رصيد الحساب

        # حساب حجم العقد بناءً على رصيد الحساب ونسبة المخاطرة
        volume = calculate_lot_size(account_balance, risk_percentage)

        # الحصول على الصفقات المفتوحة للرمز المحدد
        positions = mt5.positions_get(symbol=symbol)
        if positions is None:
            print("لا توجد صفقات مفتوحة للرمز المحدد أو حدث خطأ.")
            return

        for position in positions:
            # تحديد نوع الصفقة المعكوسة
            hedge_order_type = 1 if position.type == 0 else 0  # 1 = SELL, 0 = BUY

            # الحصول على السعر الحالي
            price = mt5.symbol_info_tick(symbol).ask if hedge_order_type == 0 else mt5.symbol_info_tick(symbol).bid

            # إعداد الطلب
            hedge_request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": volume,
                "type": hedge_order_type,
                "sl": position.sl,  # استخدام وقف الخسارة الحالي
                "tp": position.tp,  # استخدام جني الربح الحالي
                "magic": 123456,
                "comment": "Hedge order",
            }

            # إرسال الطلب
            result = mt5.order_send(hedge_request)
            if result is None:
                print("فشل في إرسال الطلب:", mt5.last_error())
            elif result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"فشل في فتح الصفقة: {result.retcode}, Error: {mt5.last_error()}")
            else:
                print(f"تم فتح صفقة هيدج بنجاح: {symbol}, حجم: {volume}")

    except Exception as e:
        print(f"حدث خطأ: {e}")
    finally:
        # التأكد من إغلاق الاتصال
        mt5.shutdown()


In [ ]:
import MetaTrader5 as mt5

def calculate_lot_size(account_balance, risk_percentage):
    """حساب حجم العقد بناءً على رصيد الحساب ونسبة المخاطرة."""
    risk_amount = account_balance * risk_percentage
    lot_size = risk_amount / 1000  # افتراض أن كل نقطة تمثل 10 وحدات
    return max(0.01, lot_size)  # الحد الأدنى لحجم العقد هو 0.01

def dynamic_hedge(symbol, risk_percentage=0.01):
    """تطبيق استراتيجية الهيدج مع حجم ديناميكي."""
    if not mt5.initialize():
        print("فشل في الاتصال بـ MetaTrader 5:", mt5.last_error())
        return

    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("فشل في استرداد معلومات الحساب.")
            return

        account_balance = account_info.balance
        volume = calculate_lot_size(account_balance, risk_percentage)

        positions = mt5.positions_get(symbol=symbol)
        if positions is None:
            print("لا توجد صفقات مفتوحة للرمز المحدد أو حدث خطأ.")
            return

        for position in positions:
            hedge_order_type = 1 if position.type == 0 else 0  # 1 = SELL, 0 = BUY

            price = mt5.symbol_info_tick(symbol).ask if hedge_order_type == 0 else mt5.symbol_info_tick(symbol).bid

            hedge_request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": volume,
                "type": hedge_order_type,
                "sl": position.sl,  # استخدام وقف الخسارة الحالي
                "tp": position.tp,  # استخدام جني الربح الحالي
                "magic": 123456,
                "comment": "Hedge order",
            }

            result = mt5.order_send(hedge_request)
            if result is None:
                print("فشل في إرسال الطلب:", mt5.last_error())
            elif result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"فشل في فتح الصفقة: {result.retcode}, Error: {mt5.last_error()}")
            else:
                print(f"تم فتح صفقة هيدج بنجاح: {symbol}, حجم: {volume}")

    except Exception as e:
        print(f"حدث خطأ: {e}")
    finally:
        mt5.shutdown()

# مثال على كيفية استدعاء الدالة
dynamic_hedge("XAUUSD", risk_percentage=0.01)
